# Battle of Neighbourhoods

# Introduction/ Business Problem

## Background: Booming consumerism of Paksitan

Pakistan is categorized as an emerging country with average age population of 24. With incresing income per capita and youth, consumerism is growing double digits and likely to continue so in coming years. Culturally, South Asians love food and like hanging out over lunch/ dinners and hence decent chunk of disposable income is spent on eating out. Secondly, with China starting multi billion dollars China Pakistan Economic Corridor (CPEC) with Pakistan, thousands of Chinese are moving to Pakistan with most settling in Karachi as its an economic hub of Pakistan. I intend to open a Chinese restaurant in the economic capital of Pakistan, Karachi. With diversified population of expatriates (mainly Chinese), government officials, businessmen and students, Chinese cusinies are likely to do well. 

## Business Problem: Where to open restaurant in Karachi

To succeed in food business its important to choose a location where there is more population and less restaurants. We intend to choose a location best suited for a new restaurant.

## Interested Audience

Target audience for the analysis are:
1) Anyone looking to open a new restaurant. 
2) Guide for foodies to find the neighbourhoods with different cuisines.
3) Helping Chinese expatriates to choose a location to live or hang out in Karachi.

# Data section

## Data Sources

1) Neighbourhoods/ Councils in Karachi to find number and type of restaurants concertaion: https://en.wikipedia.org/wiki/Towns_in_Karachi#Karachi_Towns
2) Four square API data

## How data will be used

Using Foursquare API, each council will be clustered with concentration of restaurants and cuisines. The data for neighbourhoods used will be scraped through a wikipedia site (https://en.wikipedia.org/wiki/Towns_in_Karachi#Karachi_Towns). The processing of data will help in identifying 1) Which Council has less concentration of restaurants, 2) Haveother amenities and entertainment areas such as Theater, offices etc and 3) Types of Cuisines already available. 

### First Importing required Liabraries:

In [59]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scraping data

In [49]:
url= requests.get('https://en.wikipedia.org/wiki/Towns_in_Karachi#Karachi_Towns').text
soup= BeautifulSoup(url, 'html.parser')
table=soup.find('table', class_='multicol')
data= table.find_all('a')
Councils= []

for i in range(0, len(data)):
    Councils.append(data[i].text.strip())

    
Councils

['Baldia Town',
 'Bin Qasim Town',
 'Gadap Town',
 'Gulberg Town',
 'Gulshan Town',
 'Jamshed Town',
 'Kiamari Town',
 'Korangi Town',
 'Landhi Town',
 'Liaquatabad Town',
 'Lyari Town',
 'Malir Town',
 'New Karachi Town',
 'North Nazimabad Town',
 'Orangi Town',
 'Saddar Town',
 'Shah Faisal Town',
 'SITE Town']

In [50]:
df_Khi= pd.DataFrame({"Neighbourhood":Councils})
df_Khi.head()

,Neighbourhood
0,Baldia Town
1,Bin Qasim Town
2,Gadap Town
3,Gulberg Town
4,Gulshan Town


In [202]:
df_Khi.shape

(18, 1)

### Getting coordinates

In [51]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Karachi, Pakistan'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [52]:
coordinates= [get_latlng(neighborhood) for neighborhood in df_Khi["Neighbourhood"].tolist()]
coordinates

[[24.928600000000074, 66.99470000000008],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.900600000000054, 67.04750000000007],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.90560000000005, 67.08220000000006],
 [24.927440000000047, 67.03479000000004],
 [24.958150000000046, 66.99139000000008],
 [24.90560000000005, 67.08220000000006],
 [24.893500000000074, 67.17510000000004],
 [24.90560000000005, 67.08220000000006]]

### Adding coordinates 

In [12]:
df_coords=pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

df_Khi['Latitude']=df_coords['Latitude']
df_Khi['Longitude']=df_coords['Longitude']
df_Khi.head()

,Neighbourhood,Latitude,Longitude
0,Baldia Town,24.9286,66.9947
1,Bin Qasim Town,24.9056,67.0822
2,Gadap Town,24.9056,67.0822
3,Gulberg Town,24.9056,67.0822
4,Gulshan Town,24.9056,67.0822


In [24]:
df_Khi.nunique()

Neighbourhood    18
Latitude          6
Longitude         6
dtype: int64

### Converting into csv and double checking the coordinates as Google API sometimes return wrong coordinates. In this case it is returning 6 coordinates for 18 neighbiurhoods

In [25]:
df_Khi.to_csv("df_Khi.csv", index= False)

### Opening the cleaned file, ensuring correct coordinates

In [60]:
df_khi_cleaned= pd.read_csv('df_Khi.csv')
df_khi_cleaned.head()

,Neighbourhood,Latitude,Longitude
0,Baldia Town,24.9525,66.9550
1,Bin Qasim Town,24.8596,67.4005
2,Gadap Town,25.0023,67.1321
3,Gulberg Town,24.9368,67.0760
4,Gulshan Town,24.9180,67.0971


#### Now we have 18 coordinates for 18 neighbourhoods

In [29]:
df_khi_cleaned.nunique()

Neighbourhood    18
Latitude         18
Longitude        18
dtype: int64

### Create a map of Karachi and neighbourhoods super imposed

In [61]:
latitude= 24.8607
longitude= 67.0011

In [36]:
map_khi=folium.Map(location=[latitude, longitude], zoom_start=10)

#superimposing neighbourhoods on the map

for lat,lng,label in zip(df_khi_cleaned['Latitude'], df_khi_cleaned['Longitude'],df_khi_cleaned['Neighbourhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3816cc',
    fill_opacity=0.7).add_to(map_khi)

map_khi



### Dialling in foursquare API

In [76]:
CLIENT_ID='YYJIOEE53UDP5MCOJFS2NGYSYP5CVOADZYJOZFQY0A55JGQB'
CLIENT_SECRET='KVRN03PBBM42K2DDYRV33WRJTK4PK1KL2ZBZNCXTCWTN1EXL'
VERSION= '20180605'

LIMIT=200
radius=2000

In [77]:
#Creating function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
khi_venues=getNearbyVenues(names=df_khi_cleaned['Neighbourhood'],
                              latitudes=df_khi_cleaned['Latitude'],
                              longitudes=df_khi_cleaned['Longitude'])

Baldia Town
Bin Qasim Town
Gadap Town
Gulberg Town
Gulshan Town
Jamshed Town
Kiamari Town
Korangi Town
Landhi Town
Liaquatabad Town
Lyari Town
Malir Town
New Karachi Town
North Nazimabad Town
Orangi Town
Saddar Town
Shah Faisal Town
SITE Town


In [79]:
khi_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Baldia Town,24.9525,66.9550,Quetta Al-Aziz Hotel,24.947769,66.944586,Café
1,Baldia Town,24.9525,66.9550,Ghosia Juice Corner,24.947562,66.940091,Juice Bar
2,Gadap Town,25.0023,67.1321,Shaheen Shinwari Bar B.Q & Restaurant,24.991692,67.134327,BBQ Joint
3,Gadap Town,25.0023,67.1321,Orchid Farm House,25.002048,67.125725,Forest
4,Gadap Town,25.0023,67.1321,Shinwari Grill,24.991818,67.134443,Restaurant
5,Gadap Town,25.0023,67.1321,Shaheen Shinwari,24.991680,67.123196,Restaurant
6,Gadap Town,25.0023,67.1321,Eastwood Tea House,24.985959,67.132494,Candy Store
7,Gadap Town,25.0023,67.1321,Pakistan Hotel,24.986492,67.123202,BBQ Joint
8,Gulberg Town,24.9368,67.0760,Mehmood Sweets,24.936394,67.076200,Bakery
9,Gulberg Town,24.9368,67.0760,Luckyone Mall,24.932513,67.086961,Shopping Mall


# Now Moving to Methodology and Analysis

# Methodology

The data will be anlysed by clustering the neighbourhood councils to find less concentration of restaurants while more people flow. Using Foursquare API data, the areas will be clustered according to type of restaurants, nearby venues and trending places. I will shortlist the top 3 locations to suit the restuarant opening. The best location will be the one where there are no or less number of Thai resturants while high concentration of offices and universities.

## Analyze Neighbourhoods
#### In outdoor food places, Chinese cuisine is followed by fast food and traditional cuisines.

In [125]:
# counting each category in Neighbouroods
khi_venues.groupby('Venue Category').count().sort_values('Venue', ascending= False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Fast Food Restaurant,26,26,26,26,26,26
Pizza Place,16,16,16,16,16,16
Bakery,14,14,14,14,14,14
Café,12,12,12,12,12,12
BBQ Joint,11,11,11,11,11,11
Chinese Restaurant,10,10,10,10,10,10
Restaurant,9,9,9,9,9,9
Burger Joint,9,9,9,9,9,9
Shopping Mall,7,7,7,7,7,7


In [81]:
# one hot encoding
khi_onehot = pd.get_dummies(khi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
khi_onehot['Neighborhood'] = khi_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(khi_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
khi_onehot=khi_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
khi_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)

khi_onehot

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,Beach,Bistro,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gym,Gym / Fitness Center,Historic Site,Hookah Bar,Hotel,Hotel Pool,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Market,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Other Nightlife,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Portuguese Restaurant,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Snack Place,Soccer Field,Social Club,Sporting Goods Shop,Steakhouse,Supermarket,Taco Place,Theater,Theme Park
0,Baldia Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Baldia Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Gadap Town,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Gadap Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Gadap Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Gadap Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Gadap Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Gadap Town,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Gulberg Town,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Gulberg Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [82]:
khi_top=khi_onehot.groupby('Neighbourhood').mean().reset_index()
khi_top

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,Beach,Bistro,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gym,Gym / Fitness Center,Historic Site,Hookah Bar,Hotel,Hotel Pool,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Market,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Other Nightlife,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Portuguese Restaurant,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Snack Place,Soccer Field,Social Club,Sporting Goods Shop,Steakhouse,Supermarket,Taco Place,Theater,Theme Park
0,Baldia Town,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Gadap Town,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Gulberg Town,0.05,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.100000,0.000000,0.000000,0.000000,0.000000,0.050000,0.05,0.050000,0.000000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.100000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000
3,Gulshan Town,0.00,0.00000,0.000000,0.023256,0.000000,0.000000,0.023256,0.023256,0.116279,0.000000,0.000000,0.000000,0.000000,0.046512,0.00,0.046512,0.000000,0.000000,0.023256,0.023256,0.023256,0.023256,0.000000,0.046512,0.046512,0.023256,0.00000,0.000000,0.0,0.000000,0.000000,0.093023,0.000000,0.000000,0.000000,0.000000,0.0,0.023256,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.00,0.139535,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.023256,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.023256,0.000000,0.023256
4,Jamshed Town,0.00,0.00000,0.014286,0.000000,0.

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = khi_top['Neighbourhood']

for ind in np.arange(khi_top.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(khi_top.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Gulberg Town,Bakery,Burger Joint,Fast Food Restaurant,Transportation Service,Theme Park,BBQ Joint,Bookstore,Café,Clothing Store,Convenience Store
1,Gulshan Town,Park,Department Store,Furniture / Home Store,Theme Park,Transportation Service,Falafel Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint
2,Jamshed Town,BBQ Joint,Convenience Store,Gym,Falafel Restaurant,Transportation Service,Fast Food Restaurant,Bakery,Bookstore,Burger Joint,Café
3,Korangi Town,Health & Beauty Service,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
4,Landhi Town,Cricket Ground,Transportation Service,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
5,Liaquatabad Town,Snack Place,Clothing Store,Jewelry Store,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café
6,New Karachi Town,Transportation Service,Optical Shop,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
7,North Nazimabad Town,African Restaurant,Bakery,Shopping Mall,Bookstore,Market,Gym,Food Court,Fast Food Restaurant,Diner,BBQ Joint
8,Orangi Town,Food & Drink Shop,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store,Cricket Ground
9,SITE Town,Café,Diner,Transportation Service,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Clothing Store,Convenience Store


### Creating a dataframe for Chinese restaurants

In [85]:
khi_chinese=khi_top[["Neighbourhood","Chinese Restaurant"]]
khi_chinese

,Neighbourhood,Chinese Restaurant
0,Baldia Town,0.000000
1,Gadap Town,0.000000
2,Gulberg Town,0.000000
3,Gulshan Town,0.000000
4,Jamshed Town,0.100000
5,Kiamari Town,0.000000
6,Korangi Town,0.000000
7,Landhi Town,0.000000
8,Liaquatabad Town,0.000000
9,Lyari Town,0.000000


### Using K-Means for clustering Chinese restaurants 

In [89]:
kclusters=3
khi_chinese_Clust= khi_chinese.drop(['Neighbourhood'],1)

kmeans= KMeans(n_clusters=kclusters, random_state=0).fit(khi_chinese_Clust)
kmeans.labels_

array([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2], dtype=int32)

In [104]:
Khi_CH=khi_chinese.copy()
Khi_CH["Cluster"]=kmeans.labels_
Khi_CH

,Neighbourhood,Chinese Restaurant,Cluster
0,Baldia Town,0.000000,1
1,Gadap Town,0.000000,1
2,Gulberg Town,0.000000,1
3,Gulshan Town,0.000000,1
4,Jamshed Town,0.100000,2
5,Kiamari Town,0.000000,1
6,Korangi Town,0.000000,1
7,Landhi Town,0.000000,1
8,Liaquatabad Town,0.000000,1
9,Lyari Town,0.000000,1


In [108]:
df_khi_cleaned2=df_khi_cleaned1.drop('Cluster Labels',1)
Khi_CH_joined=Khi_CH.join(df_khi_cleaned2.set_index('Neighbourhood'), on='Neighbourhood')
Khi_CH_joined

,Neighbourhood,Chinese Restaurant,Cluster,Latitude,Longitude
0,Baldia Town,0.000000,1,24.9525,66.9550
1,Gadap Town,0.000000,1,25.0023,67.1321
2,Gulberg Town,0.000000,1,24.9368,67.0760
3,Gulshan Town,0.000000,1,24.9180,67.0971
4,Jamshed Town,0.100000,2,24.8702,67.0524
5,Kiamari Town,0.000000,1,24.8788,66.8790
6,Korangi Town,0.000000,1,24.8387,67.1209
7,Landhi Town,0.000000,1,24.8406,67.1948
8,Liaquatabad Town,0.000000,1,24.9047,67.0472
9,Lyari Town,0.000000,1,24.8784,67.0103


In [109]:
Khi_CH_joined.shape

(17, 5)

In [127]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Khi_CH_joined['Latitude'], Khi_CH_joined['Longitude'], Khi_CH_joined['Neighbourhood'], Khi_CH_joined['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
Khi_CH_joined.loc[Khi_CH_joined['Cluster'] == 0]

,Neighbourhood,Chinese Restaurant,Cluster,Latitude,Longitude
12,North Nazimabad Town,0.031250,0,24.9372,67.0423
15,Saddar Town,0.016667,0,24.8532,67.0167


In [117]:
Khi_CH_joined.loc[Khi_CH_joined['Cluster'] == 1]

,Neighbourhood,Chinese Restaurant,Cluster,Latitude,Longitude
0,Baldia Town,0.0,1,24.9525,66.9550
1,Gadap Town,0.0,1,25.0023,67.1321
2,Gulberg Town,0.0,1,24.9368,67.0760
3,Gulshan Town,0.0,1,24.9180,67.0971
5,Kiamari Town,0.0,1,24.8788,66.8790
6,Korangi Town,0.0,1,24.8387,67.1209
7,Landhi Town,0.0,1,24.8406,67.1948
8,Liaquatabad Town,0.0,1,24.9047,67.0472
9,Lyari Town,0.0,1,24.8784,67.0103
10,Malir Town,0.0,1,24.8937,67.2163


In [118]:
Khi_CH_joined.loc[Khi_CH_joined['Cluster'] == 2]

,Neighbourhood,Chinese Restaurant,Cluster,Latitude,Longitude
4,Jamshed Town,0.1,2,24.8702,67.0524
16,Shah Faisal Town,0.1,2,24.8773,67.1591


## Observation:

### Most of the Chinese restaurants are concentrated in cluster 2 while cluster 1 is least concentrated. It seems cluster 1 is suitable candidate for opening a Chinese restaurant as it offers less competition. However, it is also important to see if the neighbourhoods in cluster 1 have other amenities and cuisine options to gauge the consumers' traffic. Therefore we will also cluster the city Karachi on "most common venues".

## Clustering Karachi with K-Means

In [ ]:
kclusters=5
khi_top_clust= khi_top.drop('Neighbourhood',1)

kmeans= KMeans(n_clusters= kclusters, random_state=0).fit(khi_top_clust)
kmeans.labels_

In [92]:
# merging the data frames

df_khi_cleaned1=df_khi_cleaned.drop(df_khi_cleaned.index[1])
df_khi_cleaned1

df_khi_merged=df_khi_cleaned1
df_khi_cleaned1['Cluster Labels']=kmeans.labels_
df_khi_merged=df_khi_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
df_khi_merged.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baldia Town,24.9525,66.9550,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gadap Town,25.0023,67.1321,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gulberg Town,24.9368,67.0760,1,Bakery,Burger Joint,Fast Food Restaurant,Transportation Service,Theme Park,BBQ Joint,Bookstore,Café,Clothing Store,Convenience Store
4,Gulshan Town,24.9180,67.0971,1,Park,Department Store,Furniture / Home Store,Theme Park,Transportation Service,Falafel Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint
5,Jamshed Town,24.8702,67.0524,2,BBQ Joint,Convenience Store,Gym,Falafel Restaurant,Transportation Service,Fast Food Restaurant,Bakery,Bookstore,Burger Joint,Café


In [119]:
# removing NaN values

df_khi_merged.dropna(axis=0, inplace=True)
df_khi_merged

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Gulberg Town,24.9368,67.0760,1,Bakery,Burger Joint,Fast Food Restaurant,Transportation Service,Theme Park,BBQ Joint,Bookstore,Café,Clothing Store,Convenience Store
4,Gulshan Town,24.9180,67.0971,1,Park,Department Store,Furniture / Home Store,Theme Park,Transportation Service,Falafel Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint
5,Jamshed Town,24.8702,67.0524,2,BBQ Joint,Convenience Store,Gym,Falafel Restaurant,Transportation Service,Fast Food Restaurant,Bakery,Bookstore,Burger Joint,Café
7,Korangi Town,24.8387,67.1209,1,Health & Beauty Service,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
8,Landhi Town,24.8406,67.1948,1,Cricket Ground,Transportation Service,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
9,Liaquatabad Town,24.9047,67.0472,1,Snack Place,Clothing Store,Jewelry Store,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café
12,New Karachi Town,24.9999,67.0648,1,Transportation Service,Optical Shop,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
13,North Nazimabad Town,24.9372,67.0423,0,African Restaurant,Bakery,Shopping Mall,Bookstore,Market,Gym,Food Court,Fast Food Restaurant,Diner,BBQ Joint
14,Orangi Town,24.9517,67.0023,1,Food & Drink Shop,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store,Cricket Ground
15,Saddar Town,24.8532,67.0167,1,Theater,BBQ Joint,Bakery,Café,Juice Bar,Diner,Transportation Service,Fast Food Restaurant,Bookstore,Burger Joint


In [95]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_khi_merged['Latitude'], df_khi_merged['Longitude'], df_khi_merged['Neighbourhood'], df_khi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining cluster 1

In [126]:
df_khi_merged.loc[df_khi_merged['Cluster Labels'] == 1, df_khi_merged.columns[[0]+ list(range(3, df_khi_merged.shape[1]))] ]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Gulberg Town,1,Bakery,Burger Joint,Fast Food Restaurant,Transportation Service,Theme Park,BBQ Joint,Bookstore,Café,Clothing Store,Convenience Store
4,Gulshan Town,1,Park,Department Store,Furniture / Home Store,Theme Park,Transportation Service,Falafel Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint
7,Korangi Town,1,Health & Beauty Service,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
8,Landhi Town,1,Cricket Ground,Transportation Service,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
9,Liaquatabad Town,1,Snack Place,Clothing Store,Jewelry Store,Transportation Service,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café
12,New Karachi Town,1,Transportation Service,Optical Shop,Fast Food Restaurant,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store
14,Orangi Town,1,Food & Drink Shop,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Café,Clothing Store,Convenience Store,Cricket Ground
15,Saddar Town,1,Theater,BBQ Joint,Bakery,Café,Juice Bar,Diner,Transportation Service,Fast Food Restaurant,Bookstore,Burger Joint


### Examining cluster 2

In [98]:
df_khi_merged.loc[df_khi_merged['Cluster Labels'] == 2, df_khi_merged.columns[[0]+ list(range(3, df_khi_merged.shape[1]))] ]



,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Jamshed Town,2,BBQ Joint,Convenience Store,Gym,Falafel Restaurant,Transportation Service,Fast Food Restaurant,Bakery,Bookstore,Burger Joint,Café
17,SITE Town,2,Café,Diner,Transportation Service,Theme Park,BBQ Joint,Bakery,Bookstore,Burger Joint,Clothing Store,Convenience Store


## Observations

Analyzing the data, it seems Saddar Town in cluster 0 (in data frame Khi_CH) presents the the best neighbourhood for opening a Chinese restaurant because 1) It has a less concentration of Chinese restaurants, 2) its a city centre, 3) its nearby/ most common venues include Theatre, Transportation service and several fast food & traditional cuisines. This shows Saddar town has high foot traffic from people coming to the downtown for movies and eating out.  If in such a locality, Chinese restaurant option is available to the people, chances of it getting successful are very high.

Despite espite cluster 1 (in data frame Khi_CH) the concentration of CHinese restaurant is lowest, it might not be a best neighbourhood to open a Chinese restaurant as the aras are mainly in outskirts where population belongs to low-mid income and prefer traditional cuisines and fast food. Secondly, Chinese moving to Pakistan are likely to stay in or around the city centre due to its proximity to the offices and other amenities.